### Загрузка датасета Geo Reviews Dataset 2023

Источник - https://github.com/yandex <br><br>
Датасет, содержащий 500к отзывов с яндекс карт, выложенный в открытый доступ для академических целей.

In [52]:
import re
import pandas as pd

In [61]:
file_path = "F:\ds\geo_reviews.tskv"

Преобразуем в датафрейм и посмотрим, какие данные, подоходящие для нашего проекта, можно забрать.

In [73]:
with open(file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

header_line = lines[0].strip()
headers = [
    col.split('=')[0] for col in header_line.split('\t')
]

data_lines = lines[1:]

data = [
    dict(item.split('=', 1) for item in line.strip().split('\t'))
    for line in data_lines
]

df = pd.DataFrame(data, columns=headers)
df


,address,name_ru,rating,rubrics,text
0,"Московская область, Электросталь, проспект Ленина, 29",Продукты Ермолино,5.,"Магазин продуктов;Продукты глубокой заморозки;Магазин мяса, колбас","Замечательная сеть магазинов в общем, хороший ассортимент, цены приемлемые, а главное качество на высоте!!! Спасибо тем, кто открыл сеть этих магазинчиков!!!!"
1,"Краснодар, Прикубанский внутригородской округ, микрорайон имени Петра Метальникова, улица Петра Метальникова, 26",LimeFit,1.,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я была удивлена: \n1. Хочешь что бы твой шкаф замыкался - купи замочек\n2. Ты должен предоставить свой отпечаток пальца (полнейшая дичь) \n3. Ставят подпись на договоре с клиентом по доверенности , графу с номером доверенности оставляют пустой , а на вопрос о номере доверенности говорят номер «2»\nВы серьезно? Номер 2? \nПредоставить доверенность не могут, но говорят что у них в клубе «свои» доверенности, типа особенные какие-то \nЦирк."
2,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал.\nНо иногда бывают неутоюбные ботинки и крепления для сноуборда .\nНо у меня редкий размер ноги
3,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.,"Салон красоты;Визажисты, стилисты;Салон бровей и ресниц","Топ мастер Ангелина топ во всех смыслах ) Немного волновалась перед посещением, потому что первый раз делала брови и ресницы. Ушла довольная максимально. Понравилось всё, от итога работы до отношения мастера. Всё объяснили рассказали, как проходит процедура, как ухаживать за результатом, даже щеточки дали для бровей))) Жаль уезжаю из Твери, так бы ходила только сюда! )"
4,"Иркутская область, Черемхово, Первомайская улица, 199В",Tele2,5.,Оператор сотовой связи;Интернет-провайдер,"Приятное общение, все доступно объяснили, мне понравилось"
...,...,...,...,...,...
499994,"Москва, Южный административный округ, район Бирюлёво Западное",Бирюлёво-Пассажирская,4.,Железнодорожная станция,"Охрана кривая но добрая, двери не закрываются автоматически, хотя для этого всë есть, идëт разбазаривание средств РЖД. Вестибюль в мороз очень холодный. Установленной более 7инфракрасных обогревателей но начальник станции не хозайственник и не умеет следить за вверенным ему транспортным узлом."
499995,"Москва, Южный административный округ, район Бирюлёво Западное",Бирюлёво-Пассажирская,4.,Железнодорожная станция,"По сравнению со многими современными платформами здесь ""эпоха СССР"" не закончилась. Наверное, берегут платформу, как раритет и площадку для съёмок фильмов о советском времени...\nВ общем, требует модернизации.\nНо есть и плюсы - платформы подметают от мусора, снег чистят. Сотрудники вежливые. Делают всё, что от них зависит."
499996,"Новосибирск, Коммунистическая улица, 48А",NaN,5.,"Бар, паб","Приятная атмосфера, прекрасное вино, волшебная живая музыка. Очень легко влюбиться в это место."
499997,"Астраханская область, Харабалинский район",Сарай-Бату,5.,Достопримечательность,"Был с семьёй 13.06.23 Отличное место. Рекомендую. Это конечно не настоящий город, а реконструкция и тем не менее не пожалеете. Очень реалистично! Фотки получатся хорошие! Повсюду слышна речь гида (аудио сопровождение) Цена билета 450р (дети с 7до 12 лет 200р) до 7 бесплатно. Часто встречали сусликов (не шутливые👍). За отдельную плату (500р) катают на верблюде. По дороге с трассы до самого Сарай-Бату примерно 5 км песчаного пути. Можно погонять на своей машине, как на ралли. Естественно аккуратно ✌"


Выбираем только строки, относящиеся к Москве, почистим текст от смайликов и спецсимволов, приведем к нижнему регистру.

In [74]:
df = df[df['address'].str.contains('Москва', na=False)]
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

def remove_emojis_with_replacement(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  
        "\U0001F300-\U0001F5FF"  
        "\U0001F680-\U0001F6FF"  
        "\U0001F1E0-\U0001F1FF"  
        "\U00002600-\U00002B55"  
        "\U000024C2-\U0001F251"  
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'. ', text)

df['text'] = df['text'].apply(
    remove_emojis_with_replacement
)
df['text'] = df['text'].apply(
    lambda x: re.sub(r'\\n|\\t|\\r', '', x).strip() if isinstance(x, str) else x
)

Теперь отфильтруем нужные категории организаций.

In [75]:
keywords = [
    'кафе', 'фудкорт', 'кофе',
    'бар', 'пиццерия', 'кофейня', 
    'столовая', 'ресторан', 'кофе с собой',
    'паб', 'быстрое питание', 'спортбар',
    'ночной клуб', 'кондитерская', 'суши-бар',
    'суши', 'фастфуд'
]

pattern = r'\b(?:' + '|'.join(keywords) + r')\b'

mask = df['rubrics'].str.contains(pattern, case=False, na=False)

df = df[mask]

Уберём слово "москва" из адреса, создадим уникальные ID для пар адрес + название, приведём к заголовки к виду нашего первого датасета.

In [76]:
df['address'] = df['address'].str.replace(r'^москва,\s*', '', regex=True)

df['id'] = df.groupby(['address', 'name_ru']).ngroup()

df['id'] = df['id'] + 1 
df['id'] = 'y' + df['id'].astype(str)

df = df.rename(
    columns={
        'id': 'place_id', 
        'name_ru': 'name_primary'
    }
)

df['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(0).astype(int)
df

,address,name_primary,rating,rubrics,text,place_id
7,"4-й кожевнический переулок, 4",jinju,5,кафе;кофейня,"5 из 5. пил кофе и в риме, и в париже, но вкуснее, чем капуч на фундучном молоке с фирменными сливками джинжу, не пробовал ничего! десерты тоже очень необычные. ребята - бариста большие молодцы! что можно улучшить? маловато места, с посадкой можно что-то придумать?",y298.0
8,"4-й кожевнический переулок, 4",jinju,4,кафе;кофейня,"не очень удобное расположение, от метро идти мин 20 быстрым шагом через промзону. в самом кофе мест очень мало, а желающих очень много(( пирожные очень вкусные, кофе…бывает вкуснее. второй раз именно туда на пойду.",y298.0
135,"улица остоженка, 30, стр. 1",coffee moose,5,кофейня;кофе с собой,"интересное солнечное место по пути. желтенькое. люблю все новое и интересное, решила испробовать и испить кофе. капучино восторг. вкусный ароматный бразильский и бодрящий. самое ценное персонал, внимательный, отзывчивый и тонко чувствующий. спасибо за ваш труд.",y7723.0
150,"поселение внуковское, улица анны ахматовой, 20",massimo pizza,5,ресторан;кафе,"уютное ламповое место. мне всё понравилось: расположение (легко найти), меню и ассортимент (вкусно), обслуживание (молодой коллектив). а еще там играют в мафию.",y4316.0
151,"поселение внуковское, улица анны ахматовой, 20",massimo pizza,1,ресторан;кафе,"зашли единожды в это кафе. заказали с собой еду. простите ,но так себе. картошка фри та же самая никакая. в фаст-фуд кортах и то лучше в разы. паста тоже ( будто макароны по-флоцки). вдобавок дома вскрыв коробки обнаружили, что вместо двух разных пицц нам положили две одинаковых. . пиццей в принципе это можно назвать с натяжкой.будто дешевой ветчины сверху накидали. вся еда пресная. самое отвратительно место на районе. не советую к посещению. п.с. на негативные отзывы ставят негативные лайки явно не посетители этого «кафе».",y4316.0
...,...,...,...,...,...,...
499871,"улица маросейка, 7/8",караоке-клуб маска,5,"караоке-клуб;бар, паб;банкетный зал","классное заведение,интересный интерьер и вкусная кухня)официанты отзывчивые веселые ребята,отмечали день рождение,все понравилось,круто отдохнули еще и 10% скидку в честь дня рождения сделали)приду сюда еще и не раз)",y7404.0
499984,"братиславская улица, 31, корп. 1",шаурма 1001,1,быстрое питание;кафе,"залетели ночью с друзьями, разбудили работника, сделал нас шаурму и несколько штук самсы с говядиной. 2 дня на толчке по видеосвязи обсуждали что туда больше ни ногой.",y1090.0
499987,"улица юных ленинцев, 52",petruccio. pizza&pasta,5,ресторан;кафе;пиццерия,"очень вкусная пицца. нравится их система лояльности, каждый раз приятно сбивает цену. приятное обслуживание и атмосфера. место супер.",y8353.0
499988,"улица юных ленинцев, 52",petruccio. pizza&pasta,5,ресторан;кафе;пиццерия,"пицца детям понравилась, интерьер очень интересный, заказали ещё кактейли были вкусные с красивой подачей , вежливый персонал, было все очень вкусно!",y8353.0


In [30]:
df.to_csv(r'D:\YandexDisk\Т\repo\MIPT_practice\hackathons\df_y.csv', index=False)